In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 4

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
def DRM(x):
    u_hat = model(x)
    uxx = torch.zeros(x.size()[0], dimension).cuda() 
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda() 
        dx[:, i] = torch.ones(x.size()[0])
        uxx[:, i] = (model(x+step_size*dx) - 2*model(x) + model(x-step_size*dx))[:,0]/step_size**2
    laplace_u = (torch.sum(uxx, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum((-laplace_u + pi**2*u_hat - f_temp)**2)/x.size()[0]
    
    Nb = 100
    xa1 = torch.rand(Nb, dimension).cuda()  
    xa1[:, 0] = torch.zeros(Nb)
    xa2 = torch.rand(Nb, dimension).cuda()  
    xa2[:, 0] = torch.ones(Nb)
    xb1 = torch.rand(Nb, dimension).cuda() 
    xb1[:, 1] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension).cuda() 
    xb2[:, 1] = torch.ones(Nb)
    xc1 = torch.rand(Nb, dimension).cuda() 
    xc1[:, 2] = torch.zeros(Nb)
    xc2 = torch.rand(Nb, dimension).cuda() 
    xc2[:, 2] = torch.ones(Nb)
    xd1 = torch.rand(Nb, dimension).cuda() 
    xd1[:, 3] = torch.zeros(Nb)
    xd2 = torch.rand(Nb, dimension).cuda() 
    xd2[:, 3] = torch.ones(Nb)
    dx1 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx1[:, 0] = torch.ones(xb1.size()[0])
    dx2 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx2[:, 1] = torch.ones(xb1.size()[0])
    dx3 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx3[:, 2] = torch.ones(xb1.size()[0])
    dx4 = torch.zeros(xb1.size()[0], dimension).cuda() 
    dx4[:, 3] = torch.ones(xb1.size()[0])
    uxa1 = (((model(xa1+step_size*dx1) - model(xa1-step_size*dx1))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxa2 = (((model(xa2+step_size*dx1) - model(xa2-step_size*dx1))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxb1 = (((model(xb1+step_size*dx2) - model(xb1-step_size*dx2))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxb2 = (((model(xb2+step_size*dx2) - model(xb2-step_size*dx2))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxc1 = (((model(xc1+step_size*dx3) - model(xc1-step_size*dx3))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxc2 = (((model(xc2+step_size*dx3) - model(xc2-step_size*dx3))/step_size/2 * (-1.0) - 0)**2).cuda() 
    uxd1 = (((model(xd1+step_size*dx4) - model(xd1-step_size*dx4))/step_size/2 * (1.0) - 0)**2).cuda() 
    uxd2 = (((model(xd2+step_size*dx4) - model(xd2-step_size*dx4))/step_size/2 * (-1.0) - 0)**2).cuda() 
    
    part_2 = torch.sum((uxa1 + uxa2 + uxb1 + uxb2 + uxc1 + uxc2 + uxd1 + uxd2)[:,0])/xa1.size()[0]
    lambda1 = 1.0
    return part_1 + lambda1 * part_2 / 8

In [10]:
Data_size = 2000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x.cuda()

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 8)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:0" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=4, out_features=8, bias=True)
  (linear1): Linear(in_features=8, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=8, bias=True)
  (linear4): Linear(in_features=8, out_features=8, bias=True)
  (linear5): Linear(in_features=8, out_features=8, bias=True)
  (linear6): Linear(in_features=8, out_features=8, bias=True)
  (linear_out): Linear(in_features=8, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [16]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DGM_relative_error_4D_Neumann.npy", error_save)
np.save("DGM_relative_error_4D_Neumann.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(1410.3280, device='cuda:0')
current relative error is:  tensor(2.0115, device='cuda:0')
current epoch is:  50
current loss is:  tensor(567.4990, device='cuda:0')
current relative error is:  tensor(0.8394, device='cuda:0')
current epoch is:  100
current loss is:  tensor(343.5114, device='cuda:0')
current relative error is:  tensor(0.6353, device='cuda:0')
current epoch is:  150
current loss is:  tensor(140.1287, device='cuda:0')
current relative error is:  tensor(0.4776, device='cuda:0')
current epoch is:  200
current loss is:  tensor(60.2123, device='cuda:0')
current relative error is:  tensor(0.5196, device='cuda:0')
current epoch is:  250
current loss is:  tensor(42.1089, device='cuda:0')
current relative error is:  tensor(0.4800, device='cuda:0')
current epoch is:  300
current loss is:  tensor(38.2296, device='cuda:0')
current relative error is:  tensor(0.4649, device='cuda:0')
current epoch is:  350
current loss is:  tensor(30.2164, dev

current epoch is:  3100
current loss is:  tensor(1.0898, device='cuda:0')
current relative error is:  tensor(0.1327, device='cuda:0')
current epoch is:  3150
current loss is:  tensor(1.1287, device='cuda:0')
current relative error is:  tensor(0.1260, device='cuda:0')
current epoch is:  3200
current loss is:  tensor(1.0675, device='cuda:0')
current relative error is:  tensor(0.1234, device='cuda:0')
current epoch is:  3250
current loss is:  tensor(0.9851, device='cuda:0')
current relative error is:  tensor(0.1221, device='cuda:0')
current epoch is:  3300
current loss is:  tensor(0.9180, device='cuda:0')
current relative error is:  tensor(0.1169, device='cuda:0')
current epoch is:  3350
current loss is:  tensor(0.8889, device='cuda:0')
current relative error is:  tensor(0.1197, device='cuda:0')
current epoch is:  3400
current loss is:  tensor(0.8785, device='cuda:0')
current relative error is:  tensor(0.1106, device='cuda:0')
current epoch is:  3450
current loss is:  tensor(0.8230, devic

current epoch is:  6200
current loss is:  tensor(0.1096, device='cuda:0')
current relative error is:  tensor(0.0366, device='cuda:0')
current epoch is:  6250
current loss is:  tensor(0.1456, device='cuda:0')
current relative error is:  tensor(0.0337, device='cuda:0')
current epoch is:  6300
current loss is:  tensor(0.1244, device='cuda:0')
current relative error is:  tensor(0.0389, device='cuda:0')
current epoch is:  6350
current loss is:  tensor(0.1028, device='cuda:0')
current relative error is:  tensor(0.0363, device='cuda:0')
current epoch is:  6400
current loss is:  tensor(0.1425, device='cuda:0')
current relative error is:  tensor(0.0395, device='cuda:0')
current epoch is:  6450
current loss is:  tensor(0.1013, device='cuda:0')
current relative error is:  tensor(0.0355, device='cuda:0')
current epoch is:  6500
current loss is:  tensor(0.1274, device='cuda:0')
current relative error is:  tensor(0.0310, device='cuda:0')
current epoch is:  6550
current loss is:  tensor(0.1098, devic

current epoch is:  9300
current loss is:  tensor(0.0540, device='cuda:0')
current relative error is:  tensor(0.0205, device='cuda:0')
current epoch is:  9350
current loss is:  tensor(0.0713, device='cuda:0')
current relative error is:  tensor(0.0233, device='cuda:0')
current epoch is:  9400
current loss is:  tensor(0.0568, device='cuda:0')
current relative error is:  tensor(0.0225, device='cuda:0')
current epoch is:  9450
current loss is:  tensor(0.0542, device='cuda:0')
current relative error is:  tensor(0.0197, device='cuda:0')
current epoch is:  9500
current loss is:  tensor(0.0527, device='cuda:0')
current relative error is:  tensor(0.0192, device='cuda:0')
current epoch is:  9550
current loss is:  tensor(0.0607, device='cuda:0')
current relative error is:  tensor(0.0196, device='cuda:0')
current epoch is:  9600
current loss is:  tensor(0.0554, device='cuda:0')
current relative error is:  tensor(0.0186, device='cuda:0')
current epoch is:  9650
current loss is:  tensor(0.0520, devic

current epoch is:  12350
current loss is:  tensor(0.0389, device='cuda:0')
current relative error is:  tensor(0.0149, device='cuda:0')
current epoch is:  12400
current loss is:  tensor(0.0582, device='cuda:0')
current relative error is:  tensor(0.0199, device='cuda:0')
current epoch is:  12450
current loss is:  tensor(0.0444, device='cuda:0')
current relative error is:  tensor(0.0151, device='cuda:0')
current epoch is:  12500
current loss is:  tensor(0.0446, device='cuda:0')
current relative error is:  tensor(0.0179, device='cuda:0')
current epoch is:  12550
current loss is:  tensor(0.0527, device='cuda:0')
current relative error is:  tensor(0.0200, device='cuda:0')
current epoch is:  12600
current loss is:  tensor(0.0322, device='cuda:0')
current relative error is:  tensor(0.0146, device='cuda:0')
current epoch is:  12650
current loss is:  tensor(0.0335, device='cuda:0')
current relative error is:  tensor(0.0157, device='cuda:0')
current epoch is:  12700
current loss is:  tensor(0.087

current epoch is:  15400
current loss is:  tensor(0.0528, device='cuda:0')
current relative error is:  tensor(0.0142, device='cuda:0')
current epoch is:  15450
current loss is:  tensor(0.1363, device='cuda:0')
current relative error is:  tensor(0.0217, device='cuda:0')
current epoch is:  15500
current loss is:  tensor(0.0279, device='cuda:0')
current relative error is:  tensor(0.0126, device='cuda:0')
current epoch is:  15550
current loss is:  tensor(0.0266, device='cuda:0')
current relative error is:  tensor(0.0128, device='cuda:0')
current epoch is:  15600
current loss is:  tensor(0.0268, device='cuda:0')
current relative error is:  tensor(0.0129, device='cuda:0')
current epoch is:  15650
current loss is:  tensor(0.0254, device='cuda:0')
current relative error is:  tensor(0.0121, device='cuda:0')
current epoch is:  15700
current loss is:  tensor(0.0327, device='cuda:0')
current relative error is:  tensor(0.0122, device='cuda:0')
current epoch is:  15750
current loss is:  tensor(0.028

current epoch is:  18450
current loss is:  tensor(0.0264, device='cuda:0')
current relative error is:  tensor(0.0111, device='cuda:0')
current epoch is:  18500
current loss is:  tensor(0.0319, device='cuda:0')
current relative error is:  tensor(0.0138, device='cuda:0')
current epoch is:  18550
current loss is:  tensor(0.0248, device='cuda:0')
current relative error is:  tensor(0.0121, device='cuda:0')
current epoch is:  18600
current loss is:  tensor(0.0221, device='cuda:0')
current relative error is:  tensor(0.0104, device='cuda:0')
current epoch is:  18650
current loss is:  tensor(0.0269, device='cuda:0')
current relative error is:  tensor(0.0107, device='cuda:0')
current epoch is:  18700
current loss is:  tensor(0.0231, device='cuda:0')
current relative error is:  tensor(0.0106, device='cuda:0')
current epoch is:  18750
current loss is:  tensor(0.0383, device='cuda:0')
current relative error is:  tensor(0.0137, device='cuda:0')
current epoch is:  18800
current loss is:  tensor(0.020

In [18]:
x_test = torch.rand(10, 4)
torch.abs(model(x) - u_ex(x))

tensor([[0.0206],
        [0.0311],
        [0.0146],
        ...,
        [0.0325],
        [0.0059],
        [0.0187]], device='cuda:0', grad_fn=<AbsBackward>)